In [9]:
import os
import errno
import stat
import sys

from fuse import FUSE, Operations, FuseOSError
from google.cloud import storage
from google.cloud.storage.blob import Blob
from google.api_core.exceptions import NotFound

class GCSFS(Operations):
    def __init__(self, bucket_name):
        self.client = storage.Client()
        self.bucket = self.client.bucket(bucket_name)
        self.fd = 0
        self.open_files = {}


    def create(self, path, mode):
        # Create an empty file
        blob = self.bucket.blob(path.lstrip('/'))
        blob.upload_from_string('')
        return 0
    
    def close(self, fh):
        if fh in self.open_files:
            # Perform any necessary operations to 'close' the file, if applicable
            del self.open_files[fh]

    def open(self, path, flags):
        """Open a file. This increments and returns a pseudo-file descriptor."""
        self.fd += 1
        self.open_files[self.fd] = {'path': path, 'offset': 0, 'flags': flags}
        return self.fd

    def read(self, size, fh):
        # Check if the file handle exists and get the associated path and offset
        if fh not in self.open_files:
            raise FuseOSError(errno.EBADF)  # Bad file descriptor error

        file_info = self.open_files[fh]
        path = file_info['path']
        current_offset = 0

        blob = self.bucket.blob(path.lstrip('/'))
        file_data = blob.download_as_string()

        # Read the requested data
        read_data = file_data[current_offset:current_offset + size]

        # Update the offset for this file handle
        self.open_files[fh]['offset'] += len(read_data)

        return read_data

    def mkdir(self, path, mode):
        """Create a directory (folder) at path."""
        # Append a '/' to the path if not present to signify a directory
        if not path.endswith('/'):
            path += '/'
        # Create the folder
        blob = self.bucket.blob(path.lstrip('/'))
        try:
            blob.upload_from_string('')
            return True
        except Exception as e:
            print(f"An error occurred: {e}")
            return False
        
    def opendir(self, path):
        print(path)
        """Open a directory. This is mostly for compatibility; 'opendir' itself does not track state."""
        # Ensure path ends with '/' and remove leading '/'
        if not path.endswith('/'):
            path += '/'
        path = path.lstrip('/')
        # Check if directory exists
        if self.bucket.get_blob(path) or path == "":
            self.fd += 1
            self.open_files[self.fd] = {'path': path, 'flags': os.O_RDONLY}
            return self.fd
        else:
            raise FuseOSError(errno.ENOENT)

    def readdir(self, path, fh):
        """Read a directory. Returns a list of directory contents."""
        # Ensure path ends with '/' and remove leading '/'
        if not path.endswith('/'):
            path += '/'
        path = path.lstrip('/')
        # Retrieve all blobs with the prefix of the directory
        iterator = self.bucket.list_blobs(prefix=path, delimiter='/')
        blobs = list(iterator)
        # Use iterator.prefixes to get the set of unique prefixes (subdirectories)
        prefixes = set(iterator.prefixes)
        # Get the names of all the files and directories
        contents = ['.', '..']  # Default entries
        for blob in blobs:
            name = os.path.basename(blob.name)
            if name:  # Avoid adding empty names which occur with objects that end with '/'
                contents.append(name)
        # Add the prefixes (subdirectories) as well
        for prefix in prefixes:
            contents.append(os.path.basename(prefix.rstrip('/')))
        return contents

    def remove_directory_contents(self, directory_path):
        blobs = self.bucket.list_blobs(prefix=directory_path)
        for blob in blobs:
            self.bucket.delete_blob(blob.name)
            print(f"Deleted: {blob.name}")

    def rmdir(self, path):
        directory_path = path.strip('/') + '/'  # Ensure correct directory path format
        try:
            # Check if the directory is not empty
            blobs = list(self.bucket.list_blobs(prefix=directory_path))
            if any(blob.name != directory_path for blob in blobs):
                print(f"Directory {path} is not empty, deleting all the files in the give dir")
                # Here you can decide whether to call remove_directory_contents or not
                # Uncomment the next line to remove all contents inside the directory
                self.remove_directory_contents(directory_path)
                return
#                 raise FuseOSError(errno.ENOTEMPTY)  # Directory not empty error

            # If the directory marker exists, delete it
            directory_blob = self.bucket.blob(directory_path)
            if directory_blob.exists():
                self.bucket.delete_blob(directory_blob.name)
                print(f"Deleted directory marker for {path}")
            else:
                print(f"Directory {path} does not exist or is already deleted.")
                raise FuseOSError(errno.ENOENT)  # No such file or directory error

        except NotFound:
            # The directory does not exist in GCS
            print(f"Directory {path} does not exist.")
            raise FuseOSError(errno.ENOENT)  # No such file or directory error
        except Exception as e:
            # Handle other exceptions
            print(f"Unexpected error: {e}")
            raise FuseOSError(errno.EIO)  # Input/output error

    
    def write(self, data, fh):
        # Check if the file handle exists and get the associated path and offset
        if fh not in self.open_files:
            raise FuseOSError(errno.EBADF)  # Bad file descriptor error

        file_info = self.open_files[fh]
        path = file_info['path']
        offset = file_info['offset']

        blob = self.bucket.blob(path.lstrip('/'))

        # Since GCS doesn't support partial writes, we need to read-modify-write
        existing_data = blob.download_as_string()
        # This assumes that the offset given by the write call is to be respected
        # (i.e., if the file is opened in 'append' mode, this may not be correct)
        new_data = existing_data[:offset] + data + existing_data[offset + len(data):]

        blob.upload_from_string(new_data)

        # Update the offset for this file handle
        self.open_files[fh]['offset'] += len(data)

        return len(data)


    def truncate(self, path, length, fh=None):
        blob = self.bucket.blob(path.lstrip('/'))
        file_data = blob.download_as_string()
        truncated_data = file_data[:length]
        blob.upload_from_string(truncated_data)

    def unlink(self, path):
        blob = self.bucket.blob(path.lstrip('/'))
        blob.delete()

    def rename(self, old, new):
        try:
            old_blob = self.bucket.blob(old.lstrip('/'))
            new_blob = self.bucket.blob(new.lstrip('/'))

            # Copy the old blob to the new location
            new_blob = self.bucket.copy_blob(old_blob, self.bucket, new_blob.name)

            # If the copy is successful, delete the old blob
            if new_blob:
                old_blob.delete()
            print(f"Renamed {old} to {new}.")
        except Exception as e:
            print(f"Failed to rename {old} to {new}: {e}")
            raise

In [10]:
import threading

In [11]:
mountpoint = './mnt'
bucket_name = 'fuse_ecc'

# Ensure that the mountpoint is available
if not os.path.isdir(mountpoint):
    os.makedirs(mountpoint)

In [12]:
def start_fuse(mountpoint, bucket_name):
    FUSE(GCSFS(bucket_name), mountpoint, nothreads=False, foreground=True)

In [13]:
fuse_thread = threading.Thread(target=start_fuse, args=(mountpoint, bucket_name))
fuse_thread.daemon = True  # Allows the notebook to shut down without explicitly stopping the thread
fuse_thread.start()

mount_macfuse: mount point /Users/nikhilsrirangam/Desktop/sem-3/ECC/FUSE/mnt is itself on a macFUSE volumeException in thread Thread-6:
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/python@3.9/3.9.18/Frameworks/Python.framework/Versions/3.9/lib/python3.9/threading.py", line 980, in _bootstrap_inner

    self.run()
  File "/opt/homebrew/Cellar/python@3.9/3.9.18/Frameworks/Python.framework/Versions/3.9/lib/python3.9/threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/j1/51l5z9qj33j34fr_93904_0c0000gn/T/ipykernel_25665/996168660.py", line 2, in start_fuse
  File "/opt/homebrew/lib/python3.9/site-packages/fuse.py", line 711, in __init__
    raise RuntimeError(err)
RuntimeError: 1


In [14]:
fs = GCSFS("fuse_ecc")

In [15]:
# Try to make a new directory
path = "/mnt/test/"
mode = 0o777  # Not used in this implementation, but required by the method signature
if fs.mkdir(path, mode):
    print(f"Directory {path} created successfully.")
else:
    print(f"Failed to create directory {path}.")

# Open the directory
fd = fs.opendir(path)
print(f"Directory {path} opened with fd {fd}.")

# Read the directory contents
contents = fs.readdir(path, fd)
print("Contents of directory:", contents)


Directory /mnt/test/ created successfully.
/mnt/test/
Directory /mnt/test/ opened with fd 1.
Contents of directory: ['.', '..']


In [8]:
mode = 0o777
f = "/mnt/tt/tt.txt"
# Call create on the fs instance to create the file
fs.create(f, mode)

0

In [34]:
path = "/mnt/tt/"
mode = 0o777  # Not used in this implementation, but required by the method signature
if fs.mkdir(path, mode):
    print(f"Directory {path} created successfully.")
else:
    print(f"Failed to create directory {path}.")

Directory /mnt/tt/ created successfully.


In [35]:
mode = 0o777
f = "/mnt/tt/tt.txt"
# Call create on the fs instance to create the file
fs.create(f, mode)

0

In [36]:
path = "/mnt/tt/"
fs.rmdir(path)

Directory /mnt/tt/ is not empty, deleting all the files in the give dir
Deleted: mnt/tt/
Deleted: mnt/tt/tt.txt


In [37]:
# The mode in which you want to create the file
# In actual FUSE, this would be a more complex combination of flags
mode = 0o777

path = "/mnt/tt/tt.txt"
# Call create on the fs instance to create the file
fs.create(path, mode)

# The flags indicate the operation you want to perform (e.g., os.O_WRONLY for writing only)
flags = os.O_WRONLY

# Open the file to get a file handle (fh)
fh = fs.open(path, flags)

In [38]:
# The data you want to write
data = 'Hello, FFF!'

# Write data to the file
# Here you should encode the data to bytes, and you can use the file handle if your implementation requires it
written_bytes = fs.write(data.encode(), fh)

print(f"Written {written_bytes} bytes to the file.")

Written 11 bytes to the file.


In [39]:
# Read from the file
size = 1024  # Number of bytes you want to read
offset = 0   # Offset from where you want to start reading

try:
    data_read = fs.read(size=1024, fh=fh)
except FuseOSError as e:
    print(f"Error reading file: {e}")
else:
    print(data_read)

# # Read the data using the file handle if your implementation requires it
# data_read = fs.read(size, fh)
# print(fh)
# print(data_read.decode('utf-8')) 
# print(data_read)# Assuming the data is utf-8 encoded

b'Hello, FFF!'


In [40]:
path_to_file = "/mnt/tt/tt.txt"
truncate_length = 100  # Truncate to 100 bytes
fs.truncate(path_to_file, truncate_length)
print(f"Truncated {path_to_file} to {truncate_length} bytes.")

Truncated /mnt/tt/tt.txt to 100 bytes.


In [41]:
old_path = "/mnt/tt/tt.txt"
new_path = "/mnt/tt/new.txt"
fs.rename(old_path, new_path)
print(f"Renamed {old_path} to {new_path}.")

Renamed /mnt/tt/tt.txt to /mnt/tt/new.txt.
Renamed /mnt/tt/tt.txt to /mnt/tt/new.txt.


In [24]:
path_to_file = "/mnt/tt/tt.txt"
fs.unlink(path_to_file)
print(f"Deleted {path_to_file}.")

Deleted /mnt/tt/tt.txt.


In [25]:
# Close the file handle
fs.close(fh)

In [111]:
# fs = GCSFS("fuse_ecc")

# path = '/mnt/n.txt'  # Make sure this path is within the mounted FUSE filesystem
# mode = 0o777                # File mode (e.g., 0o777 for full permissions)

# # Now, call create on the instance of the class, not on the class itself
# # The file handle (fh) is not typically provided by the caller; it's generated within the create method
# fh = fs.create(path, mode)

In [80]:
# path = '/mnt/n.txt'  # The path within the FUSE filesystem
# data = 'Hello, NIKHIL!'      # The data you want to write
# flags = os.O_WRONLY         # Flag indicating you're opening the file for writing

# # Open the file to get a file descriptor
# fh = fs.open(path, flags)

# # Write data to the file at the specified path
# # Since GCS doesn't support file descriptors, we ignore the 'fh' in the actual write call
# written_bytes = fs.write(path, data.encode(), 0, fh)

# print(f"Written {written_bytes} bytes to the file.")

In [79]:
# size = 1024                 # Number of bytes you want to read
# offset = 0                  # Offset from where you want to start reading

# data_read = fs.read(path, size, offset, fh)
# print(data_read)

In [78]:
# data_to_write = b"Hello, GCS!"
# fs.write('/mnt/yourfile.txt', data_to_write, offset=0, fh=None)

In [77]:
# data_read = fs.read('/mnt/yourfile.txt', size=1024, offset=0, fh=None)
# print(data_read)

In [76]:
# path = '/mnt/yourfile.txt'
# flags = os.O_WRONLY

# try:
#     fh = fs.open(path, flags)
#     print(f"File handle: {fh}")
# except FuseOSError as e:
#     print(f"Error opening file: {e}")